In [518]:
import math
from collections import defaultdict
probabilities = defaultdict(float)

In [519]:
# テスト
# model_file = open("../test/04-model.txt","r")
# input_file = open("../test/04-input.txt","r")

# model = model_file.readlines()
# inp = input_file.readlines()
# for line in model:
#     key,value = line.split()
#     probabilities[key] = float(value)

# 本番
training_file = open("../data/wiki-ja-train.word","r")
input_file = open("../data/wiki-ja-test.txt","r")
training = training_file.readlines()
inp = input_file.readlines()

counts = defaultdict(int)
total_count = 0
for line in training:
    words = line.split()
    words.append("</s>")
    for word in words:
        counts[word] += 1
        total_count += 1
for word,count in counts.items():
    probability = float(count)/total_count
    probabilities[unicode(word,"utf-8")] = probability

# チャレンジ
# model_file = open("../data/big-ws-model.txt","r")
# input_file = open("../data/wiki-ja-test.txt","r")
# model = model_file.readlines()
# inp = input_file.readlines()
# for line in model:
#     key,value = line.split()
#     probabilities[unicode(key,"utf-8")] = float(value)

In [520]:
best_edge = defaultdict(tuple)
best_score = defaultdict(float)
ans = ""

lambda_1 = 0.95
lambda_unknown = 1-lambda_1
V = 1000000
for line in inp:
    # 前向きステップ
    line = unicode(line.strip(),"utf-8")
    best_edge = [None for i in range(len(line)+1)]
    best_score = [0 for i in range(len(line)+1)]
    for word_end in xrange(1,len(line)+1):
        best_score[word_end] = math.pow(10,10)
        for word_begin in xrange(0,word_end):
            word = line[word_begin:word_end]
            if word in probabilities or len(word)==1:
                prob = lambda_1*float(probabilities[word])+float(1-lambda_1)/V 
                my_score = best_score[word_begin]-math.log(prob,2)
                if my_score < best_score[word_end]:
                    best_score[word_end] = my_score
                    best_edge[word_end] = (word_begin,word_end)
    # 後ろ向きステップ
    words = []
    next_edge = best_edge[len(best_edge)-1]
    while next_edge != None:
        word = line[next_edge[0]:next_edge[1]]
        words.append(word.encode("utf-8"))
        next_edge = best_edge[next_edge[0]]
    words.reverse()
    ans += " ".join(words)
    ans += "\n"
with open("../data/my_answer.word","w") as wf:
    wf.write(ans)